## Instalação Dependencias

In [ ]:
pip install PyQt5

## Classes

In [8]:
class Window:
    x_min: int
    y_min: int
    x_max: int
    y_max: int
    
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

class Viewport:
    x_min: int
    y_min: int
    x_max: int
    y_max: int
    
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        
class Ponto:
    x: int
    y: int
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Reta:
    def __init__(self, ponto1, ponto2):
        self.ponto1 = ponto1
        self.ponto2 = ponto2
        
class Poligono:
    def __init__(self, pontos):
        self.pontos = pontos 

## Transformação

In [9]:
def transformacao(viewport: Viewport, window: Window, ponto: Ponto):
    x_vp = ((ponto.x - window.x_min)/(window.x_max - window.x_min))*(viewport.x_max - viewport.x_min)
    y_vp = (1 - ((ponto.y - window.y_min)/(window.y_max - window.y_min)))*(viewport.y_max - viewport.y_min)
        
    return Ponto(x_vp, y_vp)

## Parser XML

In [24]:
import xml.etree.ElementTree as ET

def parse_xml(filename):
    # Lista para armazenar os objetos encontrados no arquivo XML
    objetos = []

    # Fazendo o parse do arquivo XML
    tree = ET.parse(filename)
    root = tree.getroot()

    # Extraindo informações da viewport
    vp_min = root.find("./viewport/vpmin")
    vp_max = root.find("./viewport/vpmax")
    viewport = Viewport(int(vp_min.attrib['x']), int(vp_min.attrib['y']), int(vp_max.attrib['x']), int(vp_max.attrib['y']))

 
    w_min = root.find("./window/wmin")
    w_max = root.find("./window/wmax")
    window = Window(float(w_min.attrib['x']), float(w_min.attrib['y']), float(w_max.attrib['x']), float(w_max.attrib['y']))

    # Extraindo informações dos pontos
    for ponto in root.findall("./ponto"):
        objetos.append(Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))
        # transformacao(viewport, window, Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))

    # Extraindo informações das retas
    for reta in root.findall("./reta"):
        ponto1 = reta.find("ponto[1]")
        ponto2 = reta.find("ponto[2]")
        objetos.append(Reta(Ponto(float(ponto1.attrib['x']), float(ponto1.attrib['y'])), Ponto(float(ponto2.attrib['x']), float(ponto2.attrib['y']))))
 # Extraindo informações dos polígonos
    for poligono in root.findall("./poligono"):
        pontos = []
        for ponto in poligono.findall("ponto"):
            pontos.append(Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))
        objetos.append(Poligono(pontos))
        
    return viewport, window, objetos

In [25]:
%matplotlib qt
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow
from PyQt5.QtGui import QPainter, QPen, QPixmap
from PyQt5.QtCore import Qt

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        # Testando a função de parser com o arquivo XML fornecido
        viewport, window, objetos = parse_xml("./input/entrada.xml")
        
        for objeto in objetos:
            if isinstance(objeto, Ponto):
                print("Ponto:")
                print(">> ", objeto.x, objeto.y)
                objeto = transformacao(viewport, window, objeto)
                print(">> ", objeto.x, objeto.y)
            if isinstance(objeto, Reta):
                    print("Reta:")
                    print(">> ", objeto.ponto1.x, objeto.ponto1.y)
                    print(">> ", objeto.ponto2.x, objeto.ponto2.y)
                    objeto.ponto1 = transformacao(viewport, window, objeto.ponto1)
                    objeto.ponto2 = transformacao(viewport, window, objeto.ponto2)
                    print(">> ", objeto.ponto1.x, objeto.ponto1.y)
                    print(">> ", objeto.ponto2.x, objeto.ponto2.y)
            if isinstance(objeto, Poligono):
                print("Poligono:")
                for ponto in objeto.pontos:
                    print(">> ", ponto.x, ponto.y)
                    ponto = transformacao(viewport, window, ponto)
                    print(">> ", ponto.x, ponto.y)
            
        
        
        # Exibindo as informações obtidas
        # print("Viewport:", viewport)
        # print("Window:", window.__dict__)
        # print("Objetos:")
        # for obj in objetos:
        #     print(type(obj).__name__, obj.__dict__)
        
        
        # self.setWindowTitle("Sei lá")
        # self.setGeometry(100, 100, 400, 400)

        # self.label = QLabel(self)
        # self.label.setGeometry(10, 10, 380, 380)

        # self.draw_line()

#     def draw_line(self):
#         pixmap = self.label.pixmap()
#         if not pixmap:
#             pixmap = QPixmap(self.label.size())
#             pixmap.fill(Qt.white)

#         painter = QPainter(pixmap)
#         pen = QPen(Qt.black)
#         pen.setWidth(2)
#         painter.setPen(pen)

#         # Coordenadas dos pontos inicial e final da reta
#         x1, y1 = 50, 50
#         x2, y2 = 350, 350

#         painter.drawLine(x1, y1, x2, y2)
# #         painter.drawLine(y2, x2, y2, x1)
# #         painter.drawLine(x1, y1, y2, x1)
#         painter.end()

#         self.label.setPixmap(pixmap)


if __name__ == "__main__":
#     app = QApplication(sys.argv)
    window = MainWindow()
#     window.show()
#     sys.exit(app.exec_())


Ponto:
>>  0.0 2.0
>>  0.0 368.0
Ponto:
>>  2.0 4.0
>>  124.0 276.0
Ponto:
>>  4.0 6.0
>>  248.0 184.0
Reta:
>>  2.0 2.0
>>  4.0 4.0
>>  124.0 368.0
>>  248.0 276.0
Reta:
>>  4.0 4.0
>>  2.0 6.0
>>  248.0 276.0
>>  124.0 184.0
Poligono:
>>  1.0 1.0
>>  62.0 414.0
>>  1.0 2.0
>>  62.0 368.0
>>  2.0 2.0
>>  124.0 368.0
>>  2.0 1.0
>>  124.0 414.0
